In [2]:
# load packages
import pickle
import numpy as np
from pyscenic.cli.utils import load_signatures
import os
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
from pyscenic.genesig import Regulon

In [23]:
sig = []
for i in range(10):
    # set path for result folder for each pyscenic run
    folder_name = '/home/yilin/hopper/result'+str(i)
    path = os.path.join(folder_name, "epi_hopper.regulons.dat")
    # take the ctx output results$i/epi_hopper.regulons.dat, load into pyscenic with
    sig.append(load_signatures(path))

In [3]:
Regulon.

pyscenic.genesig.Regulon

In [24]:
# sig is a list storing all regulons across scenic runs. We could access one regulon with
sig[0][0]

Regulon(name='AR(+)', gene2weight=frozendict({'HMGN3': 2.1619847801594143, 'NDRG2': 2.4267839454960867, 'MKS1': 2.3978637363327797, 'ARL6IP1': 5.8258224907247325, 'METTL7B': 1.9357751586264544, 'SIPA1L2': 2.3759743470881287, 'RBFOX1': 3.8095871253139513, 'AR': 1.0, 'VPS4A': 1.932562420680616, 'ESRRG': 7.762791233417934, 'IGFALS': 5.003383249088736, 'GRAMD1C': 2.205352553927921, 'HOXD8': 5.458482232851728, 'ATAD3A': 3.480485329706084, 'THNSL2': 4.351783385717534, 'EYA2': 6.9506731488209725, 'TMEM132C': 3.1552476020569102, 'ARHGEF40': 2.15499330344807, 'CSTF2T': 4.34477851671337, 'TFB1M': 2.287409737833881}), gene2occurrence=frozendict({}), transcription_factor='AR', context=frozenset({'activating', 'taipale__NR3C2_DBD_NRGNACANNNTGTNCYN.png'}), score=4.044195561873588, nes=0.0, orthologous_identity=0.0, similarity_qvalue=0.0, annotation='')

In [25]:
# access all of the TFs (across runs)
regulons_name = []
def get_name(sig):
    return list(map(lambda x: x.transcription_factor, sig))
regulons_name = list(map(get_name,sig))
regulons_name = [item for sublist in regulons_name for item in sublist]

In [26]:
# count the frequency for each regulon. The result is a dictionary with regulon name as key and frequency as value
regulons_count = Counter(regulons_name)

In [27]:
# count the number of regulons for each number of occurrence. The result is a dictionary with frequency as key and number of regulons as value
count = Counter(regulons_count.values())
count

Counter({5: 28,
         9: 22,
         1: 107,
         8: 30,
         10: 141,
         6: 14,
         2: 63,
         7: 26,
         4: 29,
         3: 40})

In [28]:
count[10]+count[9]+count[8]

193

In [29]:
# set a threshold of 80% occurrence
def filterTheDict(dictObj, callback):
    # make a new dictionary
    newDict = dict()
    # Iterate over all the items in dictionary
    for (key, value) in dictObj.items():
        # Check if item satisfies the given condition then add to new dict
        if callback((key, value)):
            newDict[key] = value
    return newDict

# the result is a new dictionary with regulons above 80% occurrence
filter_regulon_dict = filterTheDict(regulons_count, lambda elem : elem[1] >= 8)

In [30]:
# a list of regulons that meet the threshold of 80% occurrence
regulons_filter = list(filter_regulon_dict)
len(regulons_filter)

193

In [19]:
# a list of regulons that meet the threshold of 80% occurrence
regulons_filter2 = list(filter_regulon_dict)
len(regulons_filter2)

193

In [21]:
len([x for x in regulons_filter if x in regulons_filter2])

173

In [31]:
filter_sig = []
def filter_regulon(sig):
    return list(filter(lambda x:x.transcription_factor in regulons_filter,sig))
sig_filter = list(map(filter_regulon,sig))

In [32]:
def get_gene2weight(regulon,sig):
        return list(filter(lambda x:x.transcription_factor==regulon,sig))[0].gene2weight

def get_occurrence(regulon,sig):
        weight = list(filter(lambda x:x.transcription_factor==regulon,sig))[0].gene2weight
        occurence = {x: 1 for x in weight}
        return occurence

# aggregate regulons across pySCENIC runs and calculate average gene weight. The function will return a dictionary with target gene as key and average gene2weight as value for each regulon
def aggregate_regulon(regulon):
    weight = Counter()
    occurrence = Counter()
    for i in range(10):
        if regulon in list(map(lambda x:x.transcription_factor,sig[i])):
            # update the weight dictionary
            weight.update(get_gene2weight(regulon,sig[i]))
            # update the occurence
            occurrence.update(get_occurrence(regulon,sig[i]))
            avg_weight = {k: (weight[k] / occurrence[k]) for k in weight}
    return avg_weight

In [33]:
# create each of aggregated regulons with
def create_agg_regulon(regulon):
    regulon = Regulon(name=regulon, gene2weight = aggregate_regulon(regulon), transcription_factor=regulon, gene2occurrence={})
    return regulon

In [34]:
regulons = list(map(create_agg_regulon,regulons_filter))

In [36]:
# save regulons
with open('agg.regulons.dat', 'wb') as f:
    pickle.dump(regulons, f)